# Engine.py:

## - Value class constructor:

Value(self, data, _children, _op):
- self: current object itself
- data: data from the current object
- _children: tuple to save the children of the current Value
- _op: the op that produced this node, for graphviz / debugging / etc
Defines the Value class and its properties.

In [ ]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

'''EXAMPLE USAGE'''
a = Value(3.0)

### You can use Value.data to retrieve the data from the current Value object:

In [36]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

'''EXAMPLE USAGE'''
a = Value(3.0)
a.data

3.0

### You can use Value._prev to get the children of the current Value:

In [40]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

'''EXAMPLE USAGE'''
a = Value(3.0)
b = Value(4.0)
c = a + b
c._prev

{Value(data=3.0, grad=0), Value(data=4.0, grad=0)}

### You can use Value._op to get what type of operation returned the current Value:

In [41]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

'''EXAMPLE USAGE'''
a = Value(3.0)
b = Value(4.0)
c = a + b
c._op

'+'

## - Value class print function:

In [ ]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"
    
a = Value(3.0)
b = Value(2.0)
#print funtion example
print(a)
b

## - Value class add (+) operator:

In [ ]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out
    
a = Value(1.0)
b = Value(2.0)
#addition example
c = a + b
c

## - Value class multiply (*) operator:

In [ ]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    
a = Value(3.0)
b = Value(2.0)
#addition example
c = a * b
c

## - Value class power (**) operator:

(Only supports integers and floats as powers)

In [29]:
class Value:
    '''Class for Value data and its operations.'''

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out
    
a = Value(3.0)
b = 2
#addition example
c = a ** b
c

Value(data=9.0, grad=0)

# Neural_network.py: